In [1]:
# v3.classification
# 3/12/2018

import argparse
import os
import glob
import numpy as np
import cv2
import torch
import sklearn.feature_extraction.image
from torchvision.models import DenseNet

#-----helper function to split data into batches
def divide_batch(l, n): 
    for i in range(0, l.shape[0], n):  
        yield l[i:i + n,::] 

# ----- parse command line arguments
# parser = argparse.ArgumentParser(description='Make output for entire image using Unet')
# parser.add_argument('input_pattern',
#                     help="input filename pattern. try: *.png, or tsv file containing list of files to analyze",
#                     nargs="*")

# parser.add_argument('-p', '--patchsize', help="patchsize, default 256", default=256, type=int)
# parser.add_argument('-s', '--batchsize', help="batchsize for controlling GPU memory usage, default 10", default=10, type=int)
# parser.add_argument('-o', '--outdir', help="outputdir, default ./output/", default="./output/", type=str)
# parser.add_argument('-r', '--resize', help="resize factor 1=1x, 2=2x, .5 = .5x", default=1, type=float)
# parser.add_argument('-m', '--model', help="model", default="best_model.pth", type=str)
# parser.add_argument('-i', '--gpuid', help="id of gpu to use", default=0, type=int)
# parser.add_argument('-f', '--force', help="force regeneration of output even if it exists", default=False,
#                     action="store_true")
# parser.add_argument('-b', '--basepath',
#                     help="base path to add to file names, helps when producing data using tsv file as input",
#                     default="", type=str)

# args = parser.parse_args()
# #args

# if not (args.input_pattern):
#     parser.error('No images selected with input pattern')

In [9]:
OUTPUT_DIR = 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\OrganSiteClassifying\\Test1Out\\'
resize = 1
# batch_size = args.batchsize
batch_size = 100
patch_size = 256
stride_size = patch_size//2
base_path = 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\OrganSiteClassifying\\Test\\**\\'
input_pattern = '*.png'
mdlPath = 'CUP010722_densenet_best_model.pth'

In [10]:
# ----- load network
device = torch.device(0 if 0!=-2 and torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(mdlPath, map_location=lambda storage, loc: storage) #load checkpoint to CPU and then put to device https://discuss.pytorch.org/t/saving-and-loading-torch-models-on-2-machines-with-different-number-of-gpu-devices/6666

model = DenseNet(growth_rate=checkpoint["growth_rate"], block_config=checkpoint["block_config"],
                 num_init_features=checkpoint["num_init_features"], bn_size=checkpoint["bn_size"],
                 drop_rate=checkpoint["drop_rate"], num_classes=checkpoint["num_classes"]).to(device)

model.load_state_dict(checkpoint["model_dict"])
model.eval()

print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

# ----- get file list

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

files = []
# basepath = args.basepath  #
# basepath = basepath + os.sep if len(
#     basepath) > 0 else ""  # if the user supplied a different basepath, make sure it ends with an os.sep

# if len(args.input_pattern) > 1:  # bash has sent us a list of files
#     files = args.input_pattern
# elif args.input_pattern[0].endswith("tsv"):  # user sent us an input file
#     # load first column here and store into files
#     print('loading tsv file...')
#     with open(args.input_pattern[0], 'r') as f: #added encoding="ISO-8859-1" to fix UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte
#         for line in f:
#             print(line)
#             if line[0] == "#":
#                 continue
#                 print('Continue')
#             files.append(basepath + line.strip().split("\t")[0])
# else:  # user sent us a wildcard, need to use glob to find files
#     files = glob.glob(args.basepath + args.input_pattern[0])
files = glob.glob(base_path + input_pattern)

total params: 	415812


In [11]:
OUTPUT_DIR

'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\OrganSiteClassifying\\Test1Out\\'

In [12]:
# base_path + input_pattern
# files = [ 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_12033.png','E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13569.png',
#  'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13825.png']

In [13]:
OUTPUT_DIR

'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\OrganSiteClassifying\\Test1Out\\'

In [14]:
sum([6, 3])

9

In [15]:
fname.contains('Pancreas')

NameError: name 'fname' is not defined

In [16]:
# +
# ------ work on files
#print(files)
saveScore = []
from tqdm import tqdm_notebook
classPred= open(OUTPUT_DIR+"\classPred.txt","w+")
classPredScore= open(OUTPUT_DIR+"\classPredScore.txt","w+")
classScore = open(OUTPUT_DIR+"\classScore.txt","w+")
classPredDetailed = open(OUTPUT_DIR+"\classPredDetailed.txt","w+")
for fname in tqdm_notebook(files):

    fname = fname.strip()
    newfname_class = "%s/%s_class.png" % (OUTPUT_DIR, os.path.basename(fname)[0:-4])

#     print(f"working on file: \t {fname}")

    # if not args.force and os.path.exists(newfname_class):
    #     print("Skipping as output file exists")
    #     continue
    #
    # cv2.imwrite(newfname_class, np.zeros(shape=(1, 1)))

    
    io = cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
    io = cv2.resize(io, (0, 0), fx=1, fy=1)

    io_shape_orig = np.array(io.shape)
    
    #add half the stride as padding around the image, so that we can crop it away later
    io = np.pad(io, [(stride_size//2, stride_size//2), (stride_size//2, stride_size//2), (0, 0)], mode="reflect")
    
    io_shape_wpad = np.array(io.shape)
    
    #pad to match an exact multiple of unet patch size, otherwise last row/column are lost
    npad0 = int(np.ceil(io_shape_wpad[0] / patch_size) * patch_size - io_shape_wpad[0])
    npad1 = int(np.ceil(io_shape_wpad[1] / patch_size) * patch_size - io_shape_wpad[1])

    io = np.pad(io, [(0, npad0), (0, npad1), (0, 0)], mode="constant")

    arr_out = sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)
    arr_out_shape = arr_out.shape
    arr_out = arr_out.reshape(-1,patch_size,patch_size,3)

    #in case we have a large network, lets cut the list of tiles into batches
    output = np.zeros((0,checkpoint["num_classes"]))
    bbb
    for batch_arr in divide_batch(arr_out,batch_size):
        
        arr_out_gpu = torch.from_numpy(batch_arr.transpose(0, 3, 1, 2) / 255).type('torch.FloatTensor').to(device)

        # ---- get results
        output_batch = model(arr_out_gpu)

        # --- pull from GPU and append to rest of output 
        output_batch = output_batch.detach().cpu().numpy()
        output = np.append(output,output_batch,axis=0)


#     print(output.shape)
    saveScore.append(output)
#     tileclass = np.argmax(output, axis=1)
#     print(output)
#     print(tileclass)
#     asdab
#     predc,predccounts=np.unique(tileclass, return_counts=True)
#     print(f'predcounts{predccounts}')
# #     if len(predccounts)==1:
# #         score = 0
# #     else:
# #         score = predccounts[1]/sum(predccounts)
# #     classPredDetailed.write(f"working on file: \t {fname}\n")
    
# #     print(f" score: \t{score}")
#     for c,cc in zip(predc,predccounts):
#         score = cc/(output.shape[0])
#         classPredDetailed.write(f"class/count: \t{c}\t{cc}\n")
#         print(f"class/count: \t{c}\t{cc}")
#         print(f"class {c} score: \t{score}")
        


#     classScore.write(f"{score}\n")
#     print(f"predicted class argmax([cc0,cc1]):\t{predc[np.argmax(predccounts)]}")
#     classPred.write(f"{predc[np.argmax(predccounts)]}\n")
# #     classPredDetailed.write("\n")
#     print(f"predicted class score>0.5:\t{(score>0.5)*1}")
#     classPredScore.write(f"{(score>0.5)*1}\n")
    
classPred.close()
# classPredDetailed.close()
classScore.close()
classPredScore.close()
ss = np.array(saveScore)
np.savez(OUTPUT_DIR+"classPred.npz", x=ss)

e:\mvp\venv\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


e:\mvp\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function extract_patches is deprecated; The function feature_extraction.image.extract_patches has been deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(msg, category=FutureWarning)


NameError: name 'bbb' is not defined

In [ ]:
np.savez("classPred032322.npz", x=ss)

In [ ]:
arr_out.shape

In [19]:
OUTPUT_DIR+"classPred.npz"

(9, 256, 256, 3)

In [12]:
OUTPUT_DIR+"classPred.npz"

In [21]:
OUTPUT_DIR+"classPred.npz"

'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\OrganSiteClassifying\\Test1Out\\classPred.npz'

In [25]:
tileclass = np.argmax(output, axis=1)
predc,predccounts=np.unique(tileclass, return_counts=True)

In [32]:
tileclass

array([3, 0, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [26]:
predc

array([0, 2, 3], dtype=int64)

In [37]:
miss = np.setdiff1d([0, 1, 2, 3], predc)
np.where([0, 1, 2, 3]== miss)

(array([1], dtype=int64),)

In [27]:
predccounts

array([1, 3, 5], dtype=int64)

In [28]:
for c,cc in zip(predc,predccounts):
    score = cc/(output.shape[0])
    classPredDetailed.write(f"class/count: \t{c}\t{cc}\n")
    print(f"class/count: \t{c}\t{cc}")
    print(f"class {c} score: \t{score}")

class/count: 	0	1
class 0 score: 	0.1111111111111111
class/count: 	2	3
class 2 score: 	0.3333333333333333
class/count: 	3	5
class 3 score: 	0.5555555555555556


In [29]:
output.shape[0]

9

In [30]:
cc

5

In [31]:
output

array([[-1.04360604, -3.99858356, -0.06175619,  0.11613086],
       [ 0.81532151, -4.6889596 , -1.57549262,  0.39810687],
       [-0.20982164, -2.76000309, -0.73907858,  0.4184534 ],
       [-1.64460421, -3.10157084, -0.34837577,  0.02792187],
       [-1.03979635, -3.00671291, -1.17878723,  0.28247494],
       [-0.8894555 , -2.62133813, -0.22684172,  0.3633157 ],
       [-0.91725069, -1.45947456, -0.39656031, -0.52493018],
       [-1.68480289, -1.40160561, -0.03332752, -0.49797475],
       [-1.29176652, -1.9173435 ,  0.72679305, -0.27555704]])

In [51]:
# ----- load network
device = torch.device(0 if 0!=-2 and torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(mdlPath, map_location=lambda storage, loc: storage) #load checkpoint to CPU and then put to device https://discuss.pytorch.org/t/saving-and-loading-torch-models-on-2-machines-with-different-number-of-gpu-devices/6666

model = DenseNet(growth_rate=checkpoint["growth_rate"], block_config=checkpoint["block_config"],
                 num_init_features=checkpoint["num_init_features"], bn_size=checkpoint["bn_size"],
                 drop_rate=checkpoint["drop_rate"], num_classes=checkpoint["num_classes"]).to(device)

model.load_state_dict(checkpoint["model_dict"])
model.eval()

print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

# ----- get file list

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

files = []
# basepath = args.basepath  #
# basepath = basepath + os.sep if len(
#     basepath) > 0 else ""  # if the user supplied a different basepath, make sure it ends with an os.sep

# if len(args.input_pattern) > 1:  # bash has sent us a list of files
#     files = args.input_pattern
# elif args.input_pattern[0].endswith("tsv"):  # user sent us an input file
#     # load first column here and store into files
#     print('loading tsv file...')
#     with open(args.input_pattern[0], 'r') as f: #added encoding="ISO-8859-1" to fix UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte
#         for line in f:
#             print(line)
#             if line[0] == "#":
#                 continue
#                 print('Continue')
#             files.append(basepath + line.strip().split("\t")[0])
# else:  # user sent us a wildcard, need to use glob to find files
#     files = glob.glob(args.basepath + args.input_pattern[0])
files = glob.glob(base_path + input_pattern)

total params: 	415554


In [52]:
files

['E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109057_40193.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109057_40449.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109057_40705.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109057_40961.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109057_41217.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109057_41473.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109313_40449.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109313_40705.png',
 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Met\\40 II_109313_40961.png',
 'E:\\MvP\

In [53]:
# base_path + input_pattern
# files = [ 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_12033.png','E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13569.png',
#  'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13825.png']

In [54]:
# +
# ------ work on files
#print(files)
classPred= open(OUTPUT_DIR+"\classPred.txt","w+")
classPredScore= open(OUTPUT_DIR+"\classPredScore.txt","w+")
classScore = open(OUTPUT_DIR+"\classScore.txt","w+")
# classPredDetailed = open(OUTPUT_DIR+"\classPredDetailed.txt","w+")
for fname in files:

    fname = fname.strip()
    newfname_class = "%s/%s_class.png" % (OUTPUT_DIR, os.path.basename(fname)[0:-4])

    print(f"working on file: \t {fname}")

    # if not args.force and os.path.exists(newfname_class):
    #     print("Skipping as output file exists")
    #     continue
    #
    # cv2.imwrite(newfname_class, np.zeros(shape=(1, 1)))

    
    io = cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
    io = cv2.resize(io, (0, 0), fx=1, fy=1)

    io_shape_orig = np.array(io.shape)
    
    #add half the stride as padding around the image, so that we can crop it away later
    io = np.pad(io, [(stride_size//2, stride_size//2), (stride_size//2, stride_size//2), (0, 0)], mode="reflect")
    
    io_shape_wpad = np.array(io.shape)
    
    #pad to match an exact multiple of unet patch size, otherwise last row/column are lost
    npad0 = int(np.ceil(io_shape_wpad[0] / patch_size) * patch_size - io_shape_wpad[0])
    npad1 = int(np.ceil(io_shape_wpad[1] / patch_size) * patch_size - io_shape_wpad[1])

    io = np.pad(io, [(0, npad0), (0, npad1), (0, 0)], mode="constant")

    arr_out = sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)
    arr_out_shape = arr_out.shape
    arr_out = arr_out.reshape(-1,patch_size,patch_size,3)

    #in case we have a large network, lets cut the list of tiles into batches
    output = np.zeros((0,checkpoint["num_classes"]))
    
    for batch_arr in divide_batch(arr_out,batch_size):
        
        arr_out_gpu = torch.from_numpy(batch_arr.transpose(0, 3, 1, 2) / 255).type('torch.FloatTensor').to(device)

        # ---- get results
        output_batch = model(arr_out_gpu)

        # --- pull from GPU and append to rest of output 
        output_batch = output_batch.detach().cpu().numpy()
        output = np.append(output,output_batch,axis=0)


    print(output.shape)
    tileclass = np.argmax(output, axis=1)
    predc,predccounts=np.unique(tileclass, return_counts=True)
    print(f'predcounts{predccounts}')
    if len(predccounts)==1:
        score = 0
    else:
        score = predccounts[1]/sum(predccounts)
#     classPredDetailed.write(f"working on file: \t {fname}\n")
    
    print(f" score: \t{score}")
#     for c,cc in zip(predc,predccounts):
#         score = cc/(output.shape[0])
#         classPredDetailed.write(f"class/count: \t{c}\t{cc}\n")
#         print(f"class/count: \t{c}\t{cc}")
#         print(f"class {c} score: \t{score}")
        


    classScore.write(f"{score}\n")
    print(f"predicted class argmax([cc0,cc1]):\t{predc[np.argmax(predccounts)]}")
    classPred.write(f"{predc[np.argmax(predccounts)]}\n")
#     classPredDetailed.write("\n")
    print(f"predicted class score>0.5:\t{(score>0.5)*1}")
    classPredScore.write(f"{(score>0.5)*1}\n")
    
classPred.close()
# classPredDetailed.close()
classScore.close()
classPredScore.close()

working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_109057_40193.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_109057_40449.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_109057_40705.png
(9, 2)
predcounts[6 3]
 score: 	0.3333333333333333
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_109057_40961.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_109057_41217.png
(9, 

(9, 2)
predcounts[2 7]
 score: 	0.7777777777777778
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110337_39681.png
(9, 2)
predcounts[4 5]
 score: 	0.5555555555555556
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110337_39937.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110337_40193.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110337_40449.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]

(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110849_40961.png
(9, 2)
predcounts[5 4]
 score: 	0.4444444444444444
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110849_41217.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110849_41473.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_110849_41729.png
(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted class argmax([cc0,cc1]):	0
predicted clas

(9, 2)
predcounts[6 3]
 score: 	0.3333333333333333
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111361_38913.png
(9, 2)
predcounts[4 5]
 score: 	0.5555555555555556
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111361_39169.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111361_39425.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111361_39681.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]):	0
predicted clas

(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111873_37633.png
(9, 2)
predcounts[3 6]
 score: 	0.6666666666666666
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111873_37889.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111873_38145.png
(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_111873_38401.png
(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted cla

...

(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131073_18689.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131073_18945.png
(9, 2)
predcounts[2 7]
 score: 	0.7777777777777778
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131073_19201.png
(9, 2)
predcounts[6 3]
 score: 	0.3333333333333333
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131073_19457.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]

(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131585_21505.png
(9, 2)
predcounts[1 8]
 score: 	0.8888888888888888
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131585_21761.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131841_18177.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_131841_18689.png
(9, 2)
predcounts[7 2]
 score: 	0.2222222222222222
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
work

(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_132609_19457.png
(9, 2)
predcounts[4 5]
 score: 	0.5555555555555556
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_132865_16385.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_132865_16897.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_132865_17153.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\M

(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_133633_16897.png
(9, 2)
predcounts[1 8]
 score: 	0.8888888888888888
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_133633_17153.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_133633_17409.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_133633_17665.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\M

(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_134401_17665.png
(9, 2)
predcounts[9]
 score: 	0
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_134401_17921.png
(9, 2)
predcounts[8 1]
 score: 	0.1111111111111111
predicted class argmax([cc0,cc1]):	0
predicted class score>0.5:	0
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_134401_18177.png
(9, 2)
predcounts[3 6]
 score: 	0.6666666666666666
predicted class argmax([cc0,cc1]):	1
predicted class score>0.5:	1
working on file: 	 E:\MvP\FinalExperimentWithPancreas\NewRNG\Run3\DeepLearningHeatmap\Val\Met\40 II_134401_18433.png
(9, 2)
predcounts[1 8]
 score: 	0.8888888888888888
predicted class argmax([cc0,cc1]

In [39]:
sum([6, 3])

9